In [1]:
#Standard Imports
import os
import sys
import pandas as pd
import numpy as np
# from collections import defaultdict
import timeit
import argparse
import logging
import copy
import numpy.random as rnd

sys.path.append('../')

# Custom Imports
from src.chord import Chord
from src.musical_work_input import MusicalWorkInput, Harmony
from src.midi_processing import *
# from src.learning_weights import *

import src.evaluate
import src.music_functions
import src.ga_model

In [2]:
# Importing Musical Corpus
#musical_work_df = pd.read_csv("../data/test_melody_hatikvah(israel)_4_1_minor_1.mid")
#musical_corpus = []
#for i, title, meter, key, tonality, first_on_beat, melody in musical_work_df.itertuples():
#    musical_corpus.append(MusicalWorkInput(title, meter, key, tonality, first_on_beat, [int(x) for x in melody.split(',')]))

In [3]:
input_midi = "../data/test_melody_hatikvah(israel)_4_1_minor_1.mid"
#melody, tempo_interval, meter, key, tonality, first_on_beat = midi_to_array(input_midi)
melody, tempo_interval = midi_to_array_quick(input_midi)
filename = str.split(str.split(input_midi,'/')[-1],'_')
first_on_beat = int(str.split(filename[-1],'.')[0])
tonality = filename[-2]
key = int(filename[-3])
meter = int(filename[-4])
song_title = '_'.join(filename[2:-4])
musical_corpus = []
musical_corpus.append(MusicalWorkInput(song_title, meter, key, tonality, first_on_beat, list(filter(None,melody[0]))))

In [4]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [5]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [6]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [7]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [8]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [9]:
# Model
for musical_work in [musical_corpus[-1]]:

    print('*'*20)
    print('Work: {}'.format(musical_work.title))
    print('*'*20)
    
    if musical_work.tonality == 'major':
        penalties_chord_progression = penalties_chord_progression_major
        chord_vocab = chord_vocab_major
    else:
        penalties_chord_progression = penalties_chord_progression_minor
        chord_vocab = chord_vocab_minor        
    
    #Solving Model
    ga_model = src.ga_model.GAmodel(musical_work, chord_vocab, max_generation=300, population_size=200,
                                    hard_constraints=hard_constraints,
                                    soft_constraint_w_weights=soft_constraint_w_weights,
                                    chord_progression_penalties=penalties_chord_progression,
                                    mutation_probability=[0.3, 0.8])
    solution, midi_array, progress_array = ga_model.solve()

********************
Work: hatikvah(israel)
********************
0 (1.47, 27769.5)
1 (2.91, 27769.5)
2 (4.34, 27769.5)
3 (5.7, 27769.5)
4 (7.11, 27769.5)
5 (8.56, 27769.5)
6 (9.97, 27769.5)
7 (11.42, 27769.5)
8 (12.87, 27769.5)
9 (14.29, 27769.5)
10 (15.82, 27769.5)
11 (17.5, 27769.5)
12 (19.17, 27769.5)
13 (20.77, 27769.5)
14 (22.4, 27769.5)
15 (23.87, 27769.5)
16 (25.42, 27769.5)
17 (26.86, 27769.5)
18 (28.29, 27769.5)
19 (29.71, 27769.5)
20 (31.19, 27769.5)
21 (32.59, 27769.5)
22 (33.99, 27769.5)
23 (35.42, 27769.5)
24 (36.83, 27769.5)
25 (38.32, 27769.5)
26 (39.8, 27769.5)
27 (41.25, 27769.5)
28 (42.66, 27769.5)
29 (44.01, 27769.5)
30 (45.6, 27769.5)
31 (47.36, 27769.5)
32 (49.05, 27769.5)
33 (50.57, 27769.5)
34 (52.27, 27769.5)
35 (53.99, 27769.5)
36 (55.72, 27769.5)
37 (57.16, 27769.5)
38 (58.76, 27769.5)
39 (60.18, 27769.5)
40 (61.62, 27769.5)
41 (63.1, 27769.5)
42 (64.6, 27769.5)
43 (66.1, 27769.5)
44 (67.55, 27769.5)
45 (69.02, 27769.5)
46 (70.55, 27769.5)
47 (72.05, 27769.5)
